In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist, cifar10
from kaggle.api.kaggle_api_extended import KaggleApi
import cv2

# Ensure to install YOLOv8 package
# pip install ultralytics
from ultralytics import YOLO

In [ ]:
# Assuming the images are in a directory named 'images'
image_dir = 'images'
image_size = (640, 640)
images = []

for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, image_size)
    images.append(img)

images = np.array(images)


In [ ]:
# Assuming the images are in a directory structure suitable for YOLO training
image_directory = 'path/to/your/images'
labels_directory = 'path/to/your/labels'

# Example code for loading images and labels for YOLOv8
def load_images_labels(image_dir, label_dir):
    images = []
    labels = []
    for image_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, image_file.replace('.jpg', '.txt'))
        img = cv2.imread(img_path)
        with open(label_path, 'r') as file:
            label = file.readlines()
        images.append(img)
        labels.append(label)
    return images, labels

images, labels = load_images_labels(image_directory, labels_directory)


In [ ]:
# Example using CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [ ]:
# Example using MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Example: Using MNIST dataset from Keras
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and preprocess if needed for YOLOv8
x_train = np.stack((x_train,)*3, axis=-1)  # Convert to 3 channels
x_test = np.stack((x_test,)*3, axis=-1)
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')


In [ ]:
# Load the YOLOv8 model
model = YOLO('yolov8s.pt')  # or 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' for different model sizes


In [ ]:
# Assuming 'images' and 'labels' lists are ready from previous steps
# You will need to write images and labels to disk in YOLOv8 format
train_img_dir = 'yolo_train/images'
train_label_dir = 'yolo_train/labels'
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)

for i, (img, label) in enumerate(zip(images, labels)):
    img_path = os.path.join(train_img_dir, f'{i}.jpg')
    label_path = os.path.join(train_label_dir, f'{i}.txt')
    cv2.imwrite(img_path, img)
    with open(label_path, 'w') as file:
        file.writelines(label)


In [ ]:
# Train the model using your custom dataset
model.train(data='yolo_train', epochs=100, imgsz=640)


In [ ]:
# Load a pretrained YOLOv8 model and perform transfer learning
pretrained_model = YOLO('yolov8m.pt')  # Load a medium-sized pretrained YOLOv8 model

# Data augmentation example using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Example augmentation on a single image
img = images[0]  # Replace with your image
img = img.reshape((1,) + img.shape)
for batch in datagen.flow(img, batch_size=1):
    plt.figure()
    plt.imshow(batch[0].astype('uint8'))
    break  # Generate one augmented image


In [ ]:
# Evaluate the model
results = model.val()

# Make predictions on new images
new_image = 'path/to/new/image.jpg'
result = model(new_image)

# Visualize predictions
result.show()
